In [1]:
# Install all necessary libraries
!pip install -q pandas numpy scikit-learn lightgbm timm torch torchvision sentence-transformers tqdm

# Import libraries
import pandas as pd
import numpy as np
import os
import shutil # For deleting directories
from PIL import Image
from tqdm.notebook import tqdm
import requests
from io import BytesIO
import time

# Image and Text Processing
import torch
import timm
from torchvision import transforms
from sentence_transformers import SentenceTransformer

# Modeling
import lightgbm as lgb

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

print("Libraries installed and imported successfully.")

Libraries installed and imported successfully.


In [2]:

DATA_DIR = '/content/'
TEMP_IMAGE_DIR = '/content/temp_images/' # Temporary folder for batch image downloads

# Settings
MODEL_DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 1000 # You can increase this to 2000 on Colab for more speed

print(f"Using device: {MODEL_DEVICE}") # This should now say 'cuda'
print(f"Processing in batches of: {BATCH_SIZE}")

# Load the datasets from the path you uploaded them to
try:
    train_df = pd.read_csv(f'{DATA_DIR}train.csv')
    test_df = pd.read_csv(f'{DATA_DIR}test.csv')
    print("Datasets loaded successfully.")
except FileNotFoundError:
    print(f"Error: Make sure you have uploaded train.csv and test.csv to the '{DATA_DIR}' folder.")

Using device: cuda
Processing in batches of: 1000
Datasets loaded successfully.


In [3]:
# --- Initialize Models (do this only once) ---
# Image Model
IMG_MODEL_NAME = 'efficientnet_b0'
img_model = timm.create_model(IMG_MODEL_NAME, pretrained=True, num_classes=0).to(MODEL_DEVICE)
img_model.eval()
img_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Text Model
TEXT_MODEL_NAME = 'all-MiniLM-L6-v2'
text_model = SentenceTransformer(TEXT_MODEL_NAME, device=MODEL_DEVICE)


# --- Helper Functions ---
def download_batch_images(df_batch, output_dir):
    """Downloads a small batch of images."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for _, row in df_batch.iterrows():
        filepath = os.path.join(output_dir, f"{row['sample_id']}.jpg")
        if os.path.exists(filepath): continue
        try:
            response = requests.get(row['image_link'], timeout=15)
            response.raise_for_status()
            img = Image.open(BytesIO(response.content))
            img.convert('RGB').save(filepath, 'JPEG')
        except Exception:
            pass

def get_image_embedding(image_path, model, transform):
    """Generates embedding for a single image."""
    try:
        image = Image.open(image_path).convert('RGB')
        image_tensor = transform(image).unsqueeze(0).to(MODEL_DEVICE)
        with torch.no_grad():
            return model(image_tensor).cpu().numpy().flatten()
    except Exception:
        return np.zeros(1280)

def process_batch(df_batch, temp_img_dir):
    """Processes one batch of data: download, featurize, cleanup."""
    # 1. Download images for this batch
    download_batch_images(df_batch, temp_img_dir)

    # 2. Generate Image Features
    img_features = []
    for sample_id in df_batch['sample_id']:
        fpath = os.path.join(temp_img_dir, f"{sample_id}.jpg")
        img_features.append(get_image_embedding(fpath, img_model, img_transform))
    img_features_df = pd.DataFrame(np.vstack(img_features), index=df_batch.index)

    # 3. Generate Text Features
    text_features = text_model.encode(df_batch['catalog_content'].tolist(), batch_size=64)
    text_features_df = pd.DataFrame(text_features, index=df_batch.index)

    # 4. Generate Engineered Text Features
    df_batch['ipq'] = df_batch['catalog_content'].str.extract(r'\(IPQ (\d+)\)').fillna(1).astype(int)
    df_batch['text_len'] = df_batch['catalog_content'].str.len()
    engineered_feats_df = df_batch[['ipq', 'text_len']]

    # 5. Combine all features for the batch
    return pd.concat([engineered_feats_df, img_features_df, text_features_df], axis=1)

print("Helper functions and models are ready.")

model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Helper functions and models are ready.


In [4]:
# --- Initialize Models (do this only once) ---
# Image Model
IMG_MODEL_NAME = 'efficientnet_b0'
img_model = timm.create_model(IMG_MODEL_NAME, pretrained=True, num_classes=0).to(MODEL_DEVICE)
img_model.eval()
img_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Text Model
TEXT_MODEL_NAME = 'all-MiniLM-L6-v2'
text_model = SentenceTransformer(TEXT_MODEL_NAME, device=MODEL_DEVICE)


# --- Helper Functions ---
def download_batch_images(df_batch, output_dir):
    """Downloads a small batch of images."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for _, row in df_batch.iterrows():
        filepath = os.path.join(output_dir, f"{row['sample_id']}.jpg")
        if os.path.exists(filepath): continue
        try:
            response = requests.get(row['image_link'], timeout=15)
            response.raise_for_status()
            img = Image.open(BytesIO(response.content))
            img.convert('RGB').save(filepath, 'JPEG')
        except Exception:
            pass

def get_image_embedding(image_path, model, transform):
    """Generates embedding for a single image."""
    try:
        image = Image.open(image_path).convert('RGB')
        image_tensor = transform(image).unsqueeze(0).to(MODEL_DEVICE)
        with torch.no_grad():
            return model(image_tensor).cpu().numpy().flatten()
    except Exception:
        return np.zeros(1280)

def process_batch(df_batch, temp_img_dir):
    """Processes one batch of data: download, featurize, cleanup."""
    # 1. Download images for this batch
    download_batch_images(df_batch, temp_img_dir)

    # 2. Generate Image Features
    img_features = []
    for sample_id in df_batch['sample_id']:
        fpath = os.path.join(temp_img_dir, f"{sample_id}.jpg")
        img_features.append(get_image_embedding(fpath, img_model, img_transform))
    img_features_df = pd.DataFrame(np.vstack(img_features), index=df_batch.index)

    # 3. Generate Text Features
    text_features = text_model.encode(df_batch['catalog_content'].tolist(), batch_size=64)
    text_features_df = pd.DataFrame(text_features, index=df_batch.index)

    # 4. Generate Engineered Text Features
    df_batch['ipq'] = df_batch['catalog_content'].str.extract(r'\(IPQ (\d+)\)').fillna(1).astype(int)
    df_batch['text_len'] = df_batch['catalog_content'].str.len()
    engineered_feats_df = df_batch[['ipq', 'text_len']]

    # 5. Combine all features for the batch
    return pd.concat([engineered_feats_df, img_features_df, text_features_df], axis=1)

print("Helper functions and models are ready.")

Helper functions and models are ready.


In [5]:
all_train_features = []

# Split training data into batches
train_batches = np.array_split(train_df, len(train_df) // BATCH_SIZE)

# --- Start Feature Generation for TRAINING data ---
print("\n--- Starting Feature Generation for TRAINING data ---")
for i, batch in enumerate(tqdm(train_batches, desc="Training Batches")):
    try:
        batch_features = process_batch(batch, TEMP_IMAGE_DIR)
        all_train_features.append(batch_features)
    finally:
        # CRUCIAL: Clean up images for this batch to save space
        if os.path.exists(TEMP_IMAGE_DIR):
            shutil.rmtree(TEMP_IMAGE_DIR)

# --- Combine all processed batches ---
print("\nConcatenating all generated training features...")
X_train = pd.concat(all_train_features)

# Ensure column names are strings for models like LightGBM
X_train.columns = [str(col) for col in X_train.columns]

print("Training feature generation complete!")
print("Final Train Features Shape:", X_train.shape)

# --- Save the generated features to a file ---
# This is the most important step for your workflow
X_train.to_csv("X_train_features.csv", index=False)
print("\nSuccessfully saved training features to 'X_train_features.csv'")


--- Starting Feature Generation for TRAINING data ---


Training Batches:   0%|          | 0/75 [00:00<?, ?it/s]


Concatenating all generated training features...
Training feature generation complete!
Final Train Features Shape: (75000, 1666)

Successfully saved training features to 'X_train_features.csv'


In [6]:
from google.colab import files
files.download("/content/X_train_features.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>